In [ ]:
from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)

server_process = execute_shell_command(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen2.5-1.5B-Instruct --mem-fraction-static 0.8 --port 30333 --host 0.0.0.0"  # llama3
)
wait_for_server("http://localhost:30333")

SGLang provides simple api to build multi-turn conversations.

Define the prompt template.

In [ ]:
from sglang import (
    function,
    system,
    user,
    assistant,
    gen,
    set_default_backend,
    RuntimeEndpoint,
)


@function
def multi_turn_conversation(s, country: str):
    s += system("You are Qwen, created by Alibaba Cloud. You are a helpful assistant.")
    s += user(f"What is the capital of {country}?")
    s += assistant(gen("capital", max_tokens=250))
    s += user("Name an interesting building in this city.")
    s += assistant(gen("building", max_tokens=250))
    return s

Set the default backend to sglang.

In [ ]:
set_default_backend(RuntimeEndpoint("http://localhost:30333"))

Execute the function and extract the results.

In [ ]:
state = multi_turn_conversation.run(
    country="Germany",
)

for m in state.messages():
    print(m["role"], ":", m["content"])

print("#" * 50)
print(state["capital"])
print("#" * 50)
print(state["building"])